In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df=pd.read_csv('/content/yelp_tip.csv')
df.head()

,_id,user_id,business_id,text,date,compliment_count
0,66ea489ae59c7c5b6d8be1ac,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,66ea489ae59c7c5b6d8be1ad,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,66ea489ae59c7c5b6d8be1ae,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,66ea489ae59c7c5b6d8be1af,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,66ea489ae59c7c5b6d8be1b0,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [4]:
# prompt: print schema of df

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   _id               908915 non-null  object
 1   user_id           908915 non-null  object
 2   business_id       908915 non-null  object
 3   text              908901 non-null  object
 4   date              908915 non-null  object
 5   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 41.6+ MB


In [5]:
import nltk

# Download 'stopwords' and 'wordnet' from NLTK
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd

# Download resources if not already downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Sample DataFrame for demonstration (replace with your actual DataFrame)
data = {'text': ["Example sentence for preprocessing.", None, "Another example text.", 123.45]}
df = pd.DataFrame(data)

def preprocess_text(text):
    # Convert non-string types to an empty string
    if not isinstance(text, str):
        text = ''
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(words)

# Fill NaNs with empty strings before applying preprocessing
df['text'] = df['text'].fillna('')
df['cleaned_text'] = df['text'].apply(preprocess_text)

df  # Display the DataFrame to confirm results


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,cleaned_text
0,Example sentence for preprocessing.,example sentence preprocessing
1,,
2,Another example text.,another example text
3,123.45,


In [8]:
# # Define sentiment labels
# df['sentiment'] = df['compliment_count'].apply(lambda x: 'pos' if x > 0 else 'neg')

In [9]:
print(df.columns)


Index(['text', 'cleaned_text'], dtype='object')


In [10]:
if 'compliment_count' not in df.columns:
    df['compliment_count'] = 0  # or any other default value


In [11]:
# Ensure 'compliment_count' exists in the DataFrame or add it if necessary
if 'compliment_count' not in df.columns:
    df['compliment_count'] = 0  # Add with a default value, e.g., 0

# Define sentiment labels based on 'compliment_count' values
df['sentiment'] = df['compliment_count'].apply(lambda x: 'pos' if x > 0 else 'neg')


In [12]:
#Splitting the Dataset
X = df['cleaned_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Count Vectorization
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# TF-IDF Transformation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [14]:
# Train the classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

MultinomialNB()

In [15]:
# Transform the test data
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Make predictions
predicted = clf.predict(X_test_tfidf)

In [16]:
# Evaluate the model
print(confusion_matrix(y_test, predicted))
print(classification_report(y_test, predicted))

[[2]]
              precision    recall  f1-score   support

         neg       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [17]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


df = pd.read_csv('/content/yelp_tip.csv')

# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Define a function for sentiment analysis
def analyze_sentiment(text):
    # Convert the input to string if it's not already
    if not isinstance(text, str):
        text = str(text)

    score = sia.polarity_scores(text)
    return score

# Apply sentiment analysis to the text column
df['sentiment_scores'] = df['text'].apply(analyze_sentiment)

# Expand the sentiment scores into separate columns
df_sentiment = df['sentiment_scores'].apply(pd.Series)

# Combine the original DataFrame with the sentiment scores
df = pd.concat([df, df_sentiment], axis=1)

# Function to classify sentiment based on compound score
def classify_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the classification function
df['sentiment'] = df['compound'].apply(classify_sentiment)

# Display the DataFrame with sentiment analysis results
print(df[['text', 'neg', 'neu', 'pos', 'compound', 'sentiment']])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


                                                     text    neg    neu  \
0                          Avengers time with the ladies.  0.000  1.000   
1       They have lots of good deserts and tasty cuban...  0.000  0.756   
2                  It's open even when you think it isn't  0.000  1.000   
3                               Very decent fried chicken  0.000  1.000   
4                  Appetizers.. platter special for lunch  0.000  0.597   
...                                                   ...    ...    ...   
908910              Disappointed in one of your managers.  0.383  0.617   
908911                            Great food and service.  0.000  0.423   
908912                                Love their Cubans!!  0.000  0.295   
908913                            Great pizza great price  0.000  0.196   
908914                  Food is good value but a bit hot!  0.000  0.559   

          pos  compound sentiment  
0       0.000    0.0000   neutral  
1       0.244    0.4404  po